In [28]:
#!/usr/bin/env python
"""
✅ PERFECTLY FIXED for your FLORES CSV format:
Columns: id, iso_639_3, iso_15924, glottocode, variant, text, url, domain, topic, ...
NLLB Translation: PRIORITY ↔ TARGET → translation_results/
"""

import torch
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print("=" * 80)
print("NLLB-200: PRIORITY ↔ TARGET (FLORES CSV FIXED)")
print("=" * 80)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Exact language configs for your CSV
PRIORITY_LANGUAGES = ['eng', 'fra']  # Translate FROM these TO all targets
TARGET_LANGUAGES = ['jpn', 'kor', 'hin', 'dan', 'nld', 'fin', 'deu', 'ell', 
                   'ita', 'pol', 'por', 'spa', 'swe', 'cmn']  # All targets
LANGUAGE_NAMES = {'eng': 'English', 'fra': 'French', 'jpn': 'Japanese', 'kor': 'Korean', 
                 'hin': 'Hindi', 'dan': 'Danish', 'nld': 'Dutch', 'fin': 'Finnish', 
                 'deu': 'German', 'ell': 'Greek', 'ita': 'Italian', 'pol': 'Polish', 
                 'por': 'Portuguese', 'spa': 'Spanish', 'swe': 'Swedish', 'cmn': 'Mandarin'}
DEFAULT_SCRIPTS = {'eng': 'Latn', 'fra': 'Latn', 'jpn': 'Jpan', 'kor': 'Hang', 'hin': 'Deva',
                  'dan': 'Latn', 'nld': 'Latn', 'fin': 'Latn', 'deu': 'Latn', 'ell': 'Grek',
                  'ita': 'Latn', 'pol': 'Latn', 'por': 'Latn', 'spa': 'Latn', 'swe': 'Latn', 'cmn': 'Hans'}

def get_nllb_code(iso6393, script):
    """ISO → NLLB BCP-47 (cmn_Hans → zho_Hans)"""
    mapping = {'cmn': 'zho'}
    return f"{mapping.get(iso6393, iso6393)}_{script}"

# 🔧 PERFECT CSV HANDLING (your exact columns)
print("📊 Loading your FLORES dev CSV...")
df = pd.read_csv('data/df.csv')
print("✅ Columns confirmed:", list(df.columns[:12]))  # Matches your format
print("Sample:", df.iloc[0]['text'][:100], "...")

# Fixed column names from your CSV
LANG_COL = 'iso_639_3'
SCRIPT_COL = 'iso_15924'
TEXT_COL = 'text'
ID_COL = 'id'

# Load NLLB model
print("🚀 Loading NLLB-200-distilled-600M...")
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)
print("✅ Model loaded!")

# Filter by your exact columns
lang_dfs = {}
all_langs = PRIORITY_LANGUAGES + TARGET_LANGUAGES
for lang_code in all_langs:
    script = DEFAULT_SCRIPTS[lang_code]
    filtered = df[(df[LANG_COL] == lang_code) & (df[SCRIPT_COL] == script)]
    if len(filtered) > 0:
        lang_dfs[lang_code] = filtered.sort_values(ID_COL)
        print(f"  {LANGUAGE_NAMES[lang_code]} ({script}): {len(filtered)} sentences")

# Align to common IDs (dev split)
common_ids = set(lang_dfs['eng'][ID_COL])
for lang in all_langs:
    if lang in lang_dfs:
        common_ids &= set(lang_dfs[lang][ID_COL])
common_ids = sorted(list(common_ids))[:1000]  # Cap at 1000

for lang_code in lang_dfs:
    lang_dfs[lang_code] = lang_dfs[lang_code][lang_dfs[lang_code][ID_COL].isin(common_ids)].sort_values(ID_COL)
N_SAMPLES = len(common_ids)
print(f"✅ Aligned {N_SAMPLES} dev sentences across {len(lang_dfs)} languages")

def batch_translate(texts, tgt_lang, batch_size=8):
    """Efficient batch translation"""
    forced_bos_id = tokenizer.convert_tokens_to_ids(tgt_lang)
    hypotheses = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs, forced_bos_token_id=forced_bos_id, 
                max_length=128, num_beams=4
            )
        hypotheses.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
    return hypotheses

# 🎯 Output structure for your post-processing
output_dir = Path("translation_results")
priority_root = output_dir / "PRIORITY_LANGUAGES"
target_root = output_dir / "TARGET_LANGUAGES"
priority_root.mkdir(parents=True, exist_ok=True)
target_root.mkdir(parents=True, exist_ok=True)

all_translations = []

print("\n🔄 1. PRIORITY → TARGET (eng/fra → all)")
for src_lang in PRIORITY_LANGUAGES:
    src_texts = lang_dfs[src_lang][TEXT_COL].tolist()
    src_name = LANGUAGE_NAMES[src_lang]
    
    for tgt_lang_code in TARGET_LANGUAGES:
        tgt_script = DEFAULT_SCRIPTS[tgt_lang_code]
        tgt_nllb = get_nllb_code(tgt_lang_code, tgt_script)
        hyps = batch_translate(src_texts, tgt_nllb)
        
        # Save: eng_to_jpn.csv, fra_to_cmn.csv, etc.
        df_out = pd.DataFrame({
            ID_COL: common_ids,
            'source_text': src_texts,
            'target_text': hyps
        })
        fname = f"{src_lang}_to_{tgt_lang_code}.csv"
        df_out.to_csv(priority_root / fname, index=False)
        print(f"   📄 {priority_root/fname}")
        
        # Metadata
        tgt_name = LANGUAGE_NAMES[tgt_lang_code]
        for orig, trans in zip(src_texts, hyps):
            all_translations.append({
                'source_lang': src_lang, 'source_name': src_name,
                'target_lang': tgt_lang_code, 'target_name': tgt_name,
                'source_text': orig, 'target_text': trans
            })

print("\n🔄 2. TARGET → PRIORITY (all → eng/fra)")
for src_lang_code in TARGET_LANGUAGES:
    src_texts = lang_dfs[src_lang_code][TEXT_COL].tolist()
    src_name = LANGUAGE_NAMES[src_lang_code]
    
    for tgt_lang in PRIORITY_LANGUAGES:
        tgt_nllb = get_nllb_code(tgt_lang, DEFAULT_SCRIPTS[tgt_lang])
        hyps = batch_translate(src_texts, tgt_nllb)
        
        # Save: jpn_to_eng.csv, cmn_to_fra.csv, etc.
        df_out = pd.DataFrame({
            ID_COL: common_ids,
            'source_text': src_texts,
            'target_text': hyps
        })
        fname = f"{src_lang_code}_to_{tgt_lang}.csv"
        df_out.to_csv(target_root / fname, index=False)
        print(f"   📄 {target_root/fname}")
        
        # Metadata
        tgt_name = LANGUAGE_NAMES[tgt_lang]
        for orig, trans in zip(src_texts, hyps):
            all_translations.append({
                'source_lang': src_lang_code, 'source_name': src_name,
                'target_lang': tgt_lang, 'target_name': tgt_name,
                'source_text': orig, 'target_text': trans
            })

# 📋 Comprehensive metadata
metadata_df = pd.DataFrame(all_translations)
metadata_df.to_csv(output_dir / "translation_metadata.csv", index=False)
print(f"\n🎉 DONE! Total translations: {len(metadata_df):,}")
print(f"📁 {priority_root} ({len(list(priority_root.glob('*.csv')))} files)")
print(f"📁 {target_root} ({len(list(target_root.glob('*.csv')))} files)")
print(f"📋 {output_dir}/translation_metadata.csv")


NLLB-200: PRIORITY ↔ TARGET (FLORES CSV FIXED)
Device: cuda
📊 Loading your FLORES dev CSV...


C:\Users\sp_hp\AppData\Local\Temp\ipykernel_16604\2228573540.py:42: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/df.csv')


✅ Columns confirmed: ['id', 'iso_639_3', 'iso_15924', 'glottocode', 'variant', 'text', 'url', 'domain', 'topic', 'has_image', 'has_hyperlink', 'last_updated']
Sample: يق أورو سنين، اوق علمون دري فکولتس کدوکترن يونيۏرسيتس ستانفورد ݢڤعموم اکن جتومى الت دياݢنوستيک بارو  ...
🚀 Loading NLLB-200-distilled-600M...
✅ Model loaded!
  English (Latn): 997 sentences
  French (Latn): 997 sentences
  Japanese (Jpan): 997 sentences
  Korean (Hang): 997 sentences
  Hindi (Deva): 997 sentences
  Danish (Latn): 997 sentences
  Dutch (Latn): 997 sentences
  Finnish (Latn): 997 sentences
  German (Latn): 997 sentences
  Greek (Grek): 997 sentences
  Italian (Latn): 997 sentences
  Polish (Latn): 997 sentences
  Portuguese (Latn): 997 sentences
  Spanish (Latn): 997 sentences
  Swedish (Latn): 997 sentences
  Mandarin (Hans): 997 sentences
✅ Aligned 997 dev sentences across 16 languages

🔄 1. PRIORITY → TARGET (eng/fra → all)


Translating: 100%|███████████████████████████████████████████████████████████████████| 125/125 [04:39<00:00,  2.24s/it]


   📄 translation_results\PRIORITY_LANGUAGES\eng_to_jpn.csv


...

   📄 translation_results\TARGET_LANGUAGES\swe_to_fra.csv


Translating: 100%|███████████████████████████████████████████████████████████████████| 125/125 [05:25<00:00,  2.60s/it]


   📄 translation_results\TARGET_LANGUAGES\cmn_to_eng.csv


Translating: 100%|███████████████████████████████████████████████████████████████████| 125/125 [06:41<00:00,  3.21s/it]


   📄 translation_results\TARGET_LANGUAGES\cmn_to_fra.csv

🎉 DONE! Total translations: 55,832
📁 translation_results\PRIORITY_LANGUAGES (28 files)
📁 translation_results\TARGET_LANGUAGES (28 files)
📋 translation_results/translation_metadata.csv


In [21]:
# #!/usr/bin/env python
# # coding: utf-8

# # ==========================================================
# # MULTI-LANGUAGE TRANSLATION (ENG + FRA → All Targets)
# # Output: ONLY 2 CSV files inside output_judge
# # ==========================================================

# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import pandas as pd
# import time
# from datetime import datetime
# from tqdm import tqdm
# from pathlib import Path

# # ==========================================================
# # Language Configuration
# # ==========================================================

# TARGET_LANGUAGES = [
#     "jpn",  # Japanese
#     "cmn",  # Mandarin
#     "kor",  # Korean
#     "hin",  # Hindi
#     "dan",  # Danish
#     "nld",  # Dutch
#     "fin",  # Finnish
#     "deu",  # German
#     "ell",  # Greek
#     "ita",  # Italian
#     "pol",  # Polish
#     "por",  # Portuguese
#     "spa",  # Spanish
#     "swe",  # Swedish
# ]

# PRIORITY_LANGUAGES = ["eng", "fra"]

# DEFAULT_SCRIPTS = {
#     "eng": "Latn",
#     "fra": "Latn",
#     "jpn": "Jpan",
#     "kor": "Hang",
#     "hin": "Deva",
#     "dan": "Latn",
#     "nld": "Latn",
#     "fin": "Latn",
#     "deu": "Latn",
#     "ell": "Grek",
#     "ita": "Latn",
#     "pol": "Latn",
#     "por": "Latn",
#     "spa": "Latn",
#     "swe": "Latn",
#     "cmn": "Hans",
# }

# LANGUAGE_NAMES = {
#     "eng": "English",
#     "fra": "French",
#     "jpn": "Japanese",
#     "kor": "Korean",
#     "hin": "Hindi",
#     "dan": "Danish",
#     "nld": "Dutch",
#     "fin": "Finnish",
#     "deu": "German",
#     "ell": "Greek",
#     "ita": "Italian",
#     "pol": "Polish",
#     "por": "Portuguese",
#     "spa": "Spanish",
#     "swe": "Swedish",
#     "cmn": "Mandarin"
# }

# def get_nllb_code(iso_639_3, script):
#     return f"{iso_639_3}_{script}"


# # ==========================================================
# # Model Setup
# # ==========================================================

# print("=" * 80)
# print("🚀 NLLB-200-600M TRANSLATION")
# print("=" * 80)
# print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Device: {device}")

# print("\n⏳ Loading NLLB model...")
# model_name = "facebook/nllb-200-distilled-600M"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16 if device == "cuda" else torch.float32
# ).to(device)

# print("✓ Model loaded successfully!")


# # ==========================================================
# # Load Dataset
# # ==========================================================

# print("\n⏳ Loading dataset...")
# df = pd.read_csv("data/df.csv")

# # Filter language data
# lang_dfs = {}

# for lang_code in PRIORITY_LANGUAGES + TARGET_LANGUAGES:
#     script = DEFAULT_SCRIPTS[lang_code]
#     filtered = df[
#         (df["iso_639_3"] == lang_code) &
#         (df["iso_15924"] == script)
#     ].copy()

#     if len(filtered) > 0:
#         lang_dfs[lang_code] = filtered.sort_values("id")
#         print(f"✓ Loaded {len(filtered)} sentences for {LANGUAGE_NAMES[lang_code]}")

# # Ensure aligned IDs
# common_ids = set(lang_dfs["eng"]["id"]).intersection(set(lang_dfs["fra"]["id"]))
# common_ids = sorted(list(common_ids))[:1000]

# for lang_code in lang_dfs:
#     lang_dfs[lang_code] = lang_dfs[lang_code][
#         lang_dfs[lang_code]["id"].isin(common_ids)
#     ].sort_values("id")

# print(f"\n✓ Using {len(common_ids)} aligned sentences")


# # ==========================================================
# # Translation Functions
# # ==========================================================

# def batch_translate(texts, tgt_lang, batch_size=8):
#     forced_bos_id = tokenizer.convert_tokens_to_ids(tgt_lang)
#     hypotheses = []

#     for i in tqdm(range(0, len(texts), batch_size)):
#         batch = texts[i:i + batch_size]

#         inputs = tokenizer(
#             batch,
#             return_tensors="pt",
#             padding=True
#         ).to(device)

#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 forced_bos_token_id=forced_bos_id,
#                 max_length=128,
#                 num_beams=4
#             )

#         hypotheses.extend(
#             tokenizer.batch_decode(outputs, skip_special_tokens=True)
#         )

#     return hypotheses


# def time_translate(texts, tgt_lang):
#     if device == "cuda":
#         torch.cuda.empty_cache()

#     start = time.time()
#     hyps = batch_translate(texts, tgt_lang)
#     elapsed = time.time() - start

#     return hyps, elapsed


# # ==========================================================
# # Create Output Folder
# # ==========================================================

# output_dir = Path("output_judge")
# output_dir.mkdir(parents=True, exist_ok=True)
# print("\n📁 Created folder: output_judge")


# # ==========================================================
# # Translation Phase
# # ==========================================================

# sources_eng = lang_dfs["eng"]["text"].tolist()
# sources_fra = lang_dfs["fra"]["text"].tolist()

# eng_nllb = get_nllb_code("eng", DEFAULT_SCRIPTS["eng"])
# fra_nllb = get_nllb_code("fra", DEFAULT_SCRIPTS["fra"])

# eng_translations = []
# fra_translations = []




In [22]:
# # ================= ENGLISH → TARGETS =================

# print("\n" + "=" * 60)
# print("TRANSLATING FROM ENGLISH")
# print("=" * 60)

# for tgt_lang_code in TARGET_LANGUAGES:

#     tgt_script = DEFAULT_SCRIPTS[tgt_lang_code]
#     tgt_nllb = get_nllb_code(tgt_lang_code, tgt_script)

#     print(f"\n➡ English → {LANGUAGE_NAMES[tgt_lang_code]}")

#     hyps, elapsed = time_translate(sources_eng, tgt_nllb)

#     for original, translated in zip(sources_eng, hyps):
#         eng_translations.append({
#             "source_language": "eng",
#             "target_language": tgt_lang_code,
#             "original_text": original,
#             "translated_text": translated
#         })

# print("✅ English translations complete")


# # ================= FRENCH → TARGETS =================

# print("\n" + "=" * 60)
# print("TRANSLATING FROM FRENCH")
# print("=" * 60)

# for tgt_lang_code in TARGET_LANGUAGES:

#     tgt_script = DEFAULT_SCRIPTS[tgt_lang_code]
#     tgt_nllb = get_nllb_code(tgt_lang_code, tgt_script)

#     print(f"\n➡ French → {LANGUAGE_NAMES[tgt_lang_code]}")

#     hyps, elapsed = time_translate(sources_fra, tgt_nllb)

#     for original, translated in zip(sources_fra, hyps):
#         fra_translations.append({
#             "source_language": "fra",
#             "target_language": tgt_lang_code,
#             "original_text": original,
#             "translated_text": translated
#         })

# print("✅ French translations complete")


# # ==========================================================
# # Save Final CSV Files
# # ==========================================================

# eng_df = pd.DataFrame(eng_translations)
# fra_df = pd.DataFrame(fra_translations)

# eng_df.to_csv(output_dir / "english_translations.csv", index=False)
# fra_df.to_csv(output_dir / "french_translations.csv", index=False)

# print("\n" + "=" * 60)
# print("🎉 DONE!")
# print("Files saved:")
# print("📄 output_judge/english_translations.csv")
# print("📄 output_judge/french_translations.csv")
# print("=" * 60)

In [23]:
# df=pd.read_csv("output_judge/english_translations.csv")

In [24]:
# sampled_df = (
#     df.groupby("target_language", group_keys=False)
#       .apply(lambda x: x.sample(n=3, random_state=42))
# )


In [25]:
# sampled_df